# [CPSC 322]() Data Science Algorithms
[Gonzaga University](https://www.gonzaga.edu/) |
[Sophina Luitel](https://www.gonzaga.edu/school-of-engineering-applied-science/faculty/detail/sophina-luitel-phd-0dba6a9d)

---

# PA7 Decision Trees (100 pts)

## Learner Objectives
At the conclusion of this programming assignment, participants should be able to:
* Represent a tree in Python
* Implement a decision tree classifier using the TDIDT algorithm
* Select an attribute using entropy
* Extract rules from a decision tree
* Consider the effects of using different feature subsets
* (BONUS) Visualize a tree with Graphviz and DOT


## Prerequisites
Before starting this programming assignment, participants should be able to:
* Implement test-driven development
* Evaluate classifiers using train/test sets
* Understand tree representations and common tree traversal algorithms
* Understand recursion
* Tell a data science story using Jupyter Notebook
* Understand Bramer Chapters 4 (Using Decision Trees for Classification), 5 (Decision Tree Induction), and Chapter 9 (Avoiding Overfitting of Decision Trees)

## Acknowledgments
Content used in this assignment is based upon information in the following sources:
* Adapted from Dr. Gina Sprint's, Data Science Algorithms, Fall 2024
* [Mushroom Dataset](https://archive.ics.uci.edu/dataset/73/mushroom), UCI Machine Learning Repository

## Github Classroom Setup
For this assignment, you will use GitHub Classroom to create a private code repository to track code changes and submit your assignment. Open this PA7 link to accept the assignment and create a private repository for your assignment in Github classroom: https://classroom.github.com/a/crrsupbJ

Your repo, for example, will be named DataScienceAlgorithms/pa7-yourusername (where yourusername is your Github username). I highly recommend committing/pushing regularly so your work is always backed up.

## Overview and Requirements
This assignment involves implementing a decision tree classifier. It has two main parts:
1. `mysklearn`: Test and implement a general and re-usable decision tree classifier
1. Mushroom edibility classification (pa7.ipynb): Write a Jupyter Notebook that uses `mysklearn` to perform classification tasks on a mushroom dataset

I highly encourage you to design functions that are generic and re-usable for future programming assignments and data mining tasks.

Note: we are learning data science from scratch! The only non-standard Python libraries you should need to use for this assignment are `tabulate`, `numpy` (math functions, random number generation, etc.), and `scipy` (sparingly). This means that beyond these libraries, you should not `pip install` any additional libraries beyond what is included in the [continuumio/anaconda3:2024.06-1](https://hub.docker.com/r/continuumio/anaconda3) Docker image and you should not use `pandas/sklearn/`etc... (exceptions are made for testing purposes only!!).

## Part 1: `mysklearn` (65 pts)
Our decision tree classifier we are going to implement will:
* Be constructed/stored using the nested list representation described in class
* Use the entropy-based attribute selection method described in class
* Use the majority voting method to deal with clashes described in class
* Only work with categorical attributes (you will not need to use any continuous attributes with your decision tree classifier in this assignment)

### Step 1: Implement Decision Tree Unit Tests for `myclassifiers.py`
Finish the decision tree unit tests in `test_myclassifiers.py` for `MyDecisionTreeClassifier` (Class API design inspiration: Sci-kit Learn's [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html))
1. `fit(X_train, y_train)`
    1. Finish the test function `test_decision_tree_classifier_fit()` by implementing the following test cases
        1. Use the 14 instance "interview" training set example traced in class, asserting against the tree constructed in [B Attribute Selection (Entropy) Lab Task #1](https://github.com/DataScienceAlgorithms/M5_DecisionTrees/blob/main/B%20Attribute%20Selection.ipynb)
        1. Use the 15 instance "iPhone" training set example from LA7, asserting against the tree you create with a desk check
            * Note: this dataset has clashes in it and when resolving the clashes you will have ties with majority voting... in this case, choose the class label that comes alphabetically first in the attribute domain, that way we all have the same solution tree (as opposed to using a random number generator and flipping a coin, which introduces complexity related to seeding and consistency across solutions)
1. `predict(X_test)`
    1. Finish the test function `test_decision_tree_classifier_predict()` by implementing the following test cases
        1. Use the 14 instance "interview" training set example traced in class, asserting against the two predictions made in [B Attribute Selection (Entropy) Lab Task #2](https://github.com/DataScienceAlgorithms/M5_DecisionTrees/blob/main/B%20Attribute%20Selection.ipynb)
        1. For the 15 instance "iPhone" training set example from LA7, use the same two unseen instances from LA7, asserting against the solution predictions from your own desk check
        
For convenience, I've provided the "interview" dataset as Python lists below.

In [1]:
# interview dataset
header_interview = ["level", "lang", "tweets", "phd", "interviewed_well"]
X_train_interview = [
    ["Senior", "Java", "no", "no"],
    ["Senior", "Java", "no", "yes"],
    ["Mid", "Python", "no", "no"],
    ["Junior", "Python", "no", "no"],
    ["Junior", "R", "yes", "no"],
    ["Junior", "R", "yes", "yes"],
    ["Mid", "R", "yes", "yes"],
    ["Senior", "Python", "no", "no"],
    ["Senior", "R", "yes", "no"],
    ["Junior", "Python", "yes", "no"],
    ["Senior", "Python", "yes", "yes"],
    ["Mid", "Python", "no", "yes"],
    ["Mid", "Java", "yes", "no"],
    ["Junior", "Python", "no", "yes"]
]
y_train_interview = ["False", "False", "True", "True", "True", "False", "True", "False", "True", "True", "True", "True", "True", "False"]

# note: this tree uses the generic "att#" attribute labels because fit() does not and should not accept attribute names
# note: the attribute values are sorted alphabetically
tree_interview = \
        ["Attribute", "att0",
            ["Value", "Junior", 
                ["Attribute", "att3",
                    ["Value", "no", 
                        ["Leaf", "True", 3, 5]
                    ],
                    ["Value", "yes", 
                        ["Leaf", "False", 2, 5]
                    ]
                ]
            ],
            ["Value", "Mid",
                ["Leaf", "True", 4, 14]
            ],
            ["Value", "Senior",
                ["Attribute", "att2",
                    ["Value", "no",
                        ["Leaf", "False", 3, 5]
                    ],
                    ["Value", "yes",
                        ["Leaf", "True", 2, 5]
                    ]
                ]
            ]
        ]

### Step 2: `fit()` and `predict()`
Complete the `mysklearn.myclassifiers.MyDecisionTreeClassifier` methods `fit()` and `predict()` and test your code for functional correctness against the above unit tests.

### Step 3: `print_decision_rules()`
Finish the `print_decision_rules()` method of `MyDecisionTreeClassifier` that prints out the rules inferred from a decision tree created from a call to `fit()`. Your rules should take the form:

```
IF att0 == val AND ... THEN class = label
IF att1 == val AND ... THEN class = label
...
```

Where "att0", "att1", ... and "class" are replaced with the contextual attribute names and class name if the keyword argument `attribute_names` is not None.

## Part 2: Mushroom Classification (25 pts)

Create a decision tree classifier for a Mushroom dataset. I extracted the dataset from here: https://archive.ics.uci.edu/dataset/73/mushroom. The dataset has been reduced to 500 instances for simplicity and only includes a subset of 9 predictive features, with each feature’s values simplified to at most 3 categories.

Some notes:
* The dataset has been sampled and slightly modified from the original to make it suitable for this assignment.
* Each feature is categorical and has been simplified for easier interpretation.
* The target class is `label`, which indicates whether a mushroom is edible or poisonous.

The features in this dataset are:
* cap-color: gray, brown, other
* odor: foul, none, other
* stalk-surface-above-ring: smooth, silky, other
* stalk-surface-below-ring: smooth, silky, other
* stalk-color-above-ring: pink, white, other
* stalk-color-below-ring: pink, white, other
* ring-type: pendant, evanescent, other
* population: several, other
* habitat: wood, grass, other

The target class (to predict) is:
label:  edible or poisonous

Since each feature is categorical, you do not need to perform any discretization, etc. The following two steps below have you exploring different subsets. For each step, create decision tree classifier to predict the label of mushroom. Test your classifier using k-fold cross-validation (with k = 10, use stratified k-fold cross validation if you implemented it for the PA5 bonus). Format your results as per PA6 and compare your results using:
1. Accuracy and error rate
1. Precision, recall, and F1 measure
1. Confusion matrices
   
### Step 1: Using only the Odor Feature

This provides a baseline set of results. It shows how accurately the model can predict whether a mushroom is edible or poisonous when using only the odor feature.

### Step 2: Try 3–4 small feature subsets (2–5 features each). 

Example subsets:  
Subset 1: odor, cap-color  
Subset 2: odor, stalk-surface-above, stalk-color-above  
Subset 3: odor, ring-type, spore-print-color  
Subset 4: odor, cap-color, stalk-color-below, population, habitat  

Run 10-fold CV for each subset and compare performance. Identify the subset that gives the best predictive performance.  

Notes:
1. You do not need to try all possible combinations, just pick some subsets.
2. Because decision trees tend to overfit, start with only 2–5 features; otherwise, your decision rules may become long and hard to interpret.
3. Odor is already a strong predictor in this dataset, so adding more features may not dramatically improve accuracy. However, feel free to experiment with combinations like habitat, ring-type, or population to see if your accuracy or tree simplicity improves.


Lastly, print out the rules inferred from your decision tree classifiers when trained over the entire dataset (as opposed to the cross validation trees) with your "best" feature subset. Based on the rules, determine ways your trees can/should be pruned. Note you do not need to write code to perform pruning, just explain how they can be pruned and give the resulting "pruned" rules

## BONUS (8 pts)
Finish the `visualize_tree()` method of `MyDecisionTreeClassifier` that generates a Graphviz .dot file and a .pdf file. The .dot file is used to produce a visual PDF representation of the decision tree. The tree should have unique nodes for all attributes and leaves in the tree (this means no node should have more than one incoming edge in the visualization). You can read more about dot and how to do this in the [D Tree Visualization and Pruning](https://github.com/DataScienceAlgorithms/M5_DecisionTrees/blob/main/D%20Tree%20Visualization%20and%20Pruning.ipynb) notes on Github. Call this method for the tree fit for part 2 step 2 (e.g. the tree used to print the decision rules). 

Include the graph generated as a PDF file (produced via dot) in your repo in a directory called `tree_vis`.

## Submitting Assignments
1. Turn in your assignment files via a Github Classroom repo. See the "Github Classroom Setup" section at the beginning of this document for details on how to do this.
    1. Your repo should contain all of the files needed to run and test your solution (e.g. .py file(s), input files, etc.). 
    1. Double-check that this is the case by "pretending to be the grader": clone (or download a zip) your submission repo and run your code in a fresh [continuumio/anaconda3:2024.06-1](https://hub.docker.com/r/continuumio/anaconda3) Docker container like we will when we grade your code.
1. Submit this PA’s associated assignment in Canvas to mark your PA as "done" and ready for grading. We will then pull your Github repo and grade your PA as soon as possible. The date and time you submit the PA assignment in Canvas will be used for marking your assignment as "late" or "on-time."

## Grading Guidelines
This assignment is worth 100 points + 8 points bonus. Your assignment will be evaluated based on a successful execution in the [continuumio/anaconda3:2024.06-1](https://hub.docker.com/r/continuumio/anaconda3) Docker container and adherence to the program requirements. We will grade according to the following criteria:
* 20 pts for correct part 1 step 1 (finish iPhone dataset tree and define unit tests)
* 25 pts for correct part 1 step 2 (finish `MyDecisionTreeClassifier` and pass `fit()` test)
* 10 pts for correct part 1 step 2 (finish `MyDecisionTreeClassifier` and pass `predict()` test)
* 10 pts for correct part 1 step 3 (finish `print_decision_rules()`)
* 10 pts for correct part 2 (mushroom classification algorithm comparison and evaluation)
* 10 pts for correct part 2 (mushroom classification feature subset comparison and evaluation)
* 5 pts for correct part 2 (mushroom classification printing of decision rules and pruning commentary)
*  10 pts for adherence to course [coding standard](https://github.com/DataScienceAlgorithms/PAs/blob/main/Coding%20Standard.ipynb), including data storytelling (narrative is clear and grammatically correct, Notebook is organized with headers and formulas are typeset with Latex)
    